In [2]:
import pandas as pd
import os
import numpy as np
from math import isnan

In [3]:
# read in gene reference and remove unformatted genes
gene = pd.read_table('/share/fsmresfiles/SpatialT/drug-target-results/HGNC.tsv')
gene = gene[['NCBI Gene ID(supplied by NCBI)','Approved symbol']]
gene.columns = ['num','name']
lst = []
for i in gene['num'].tolist():
    if pd.isna(i) != True:
        lst.append(int(i))
    else:
        lst.append(i)
gene['num'] = lst
gene_dict = gene.set_index('num').to_dict()['name']

In [4]:
clean_hgnc_dict = filter(lambda k: not isnan(k), gene_dict)
clean_hgnc_dict = {k: gene_dict[k] for k in gene_dict if not isnan(k)}
clean_hgnc_dict = {int(k):v for k,v in clean_hgnc_dict.items()}

In [5]:
# interactome for ppi network
interactome = pd.read_csv('/share/fsmresfiles/SpatialT/drug-target-results-OVR-0725/Analysis_PPI/interactome.tsv', sep='\t')

In [6]:
interactome

,#Protein A,Protein B,Source,Type
0,1,310,"InWeb_IM,IntAct,PINA",Literature
1,1,368,"InWeb_IM,IntAct,PINA",Literature
2,1,1026,"InWeb_IM,IntAct,PINA",Literature
3,1,2232,BioPlex2.0,Complex
4,1,2886,"InWeb_IM,IntAct,PINA",Literature
...,...,...,...,...
351439,100506164,100506164,IntAct,Literature
351440,100506658,100506658,HPRD,Literature
351441,100506742,100506742,BioGRID,Literature
351442,100532737,100532737,DFCI_NET_2016,Binary


In [7]:
unmappable_pro = []
Protein_A_Name = []
Protein_B_Name = []

for i in interactome['#Protein A'].tolist():
    if i in clean_hgnc_dict.keys(): 
        Protein_A_Name.append(clean_hgnc_dict[i] )
    else:
        if i not in unmappable_pro: 
            unmappable_pro.append(i)
            print(i)
        Protein_A_Name.append('drop')

for i in interactome['Protein B'].tolist():
    if i in clean_hgnc_dict.keys(): 
        Protein_B_Name.append(clean_hgnc_dict[i] )
    else:
        if i not in unmappable_pro: 
            unmappable_pro.append(i)
            print(i)
        Protein_B_Name.append('drop')

244
4029
5171
7795
9503
10638
11217
23285
26148
55747
79686
80761
114299
142937
153684
155060
285464
389834
400590
554045
643752
283970
728932
338809
445347
554223
729603
727738
100294715
441520
1283
100287428
653219
221016
348738
101243544
100507714
85371
440157
642947
339457
164022
653252
100129534
728392
252842
6025
145757
283011
497048
100127889
100652775
728461
653973
100129885
414927
730441
117153
728071
374467
388289
389523
641590
654348
728936
729368
646938
100862685
284083
445341
653602
100526832
653129
284912
728013
162137
643832
442028
497049
727751
392490
554206
652276


In [8]:
# drop unmappable interactome interactions
interactome['Protein_A_Name'] = Protein_A_Name
interactome['Protein_B_Name'] = Protein_B_Name
dropa_index = interactome[interactome['Protein_A_Name'] == 'drop'].index.tolist()
dropb_index = interactome[interactome['Protein_B_Name'] == 'drop'].index.tolist()
print(len(dropa_index), len(dropb_index))
# 538 interactions removed due to unmappable gene number 
for i in dropa_index:
    if i not in dropb_index:
        dropb_index.append(i)
len(dropb_index)

101 441


538

In [9]:
clean_interactome = interactome.drop(dropb_index)

In [16]:
# sample level ppi
#for idx, row in all_patho_deg.iterrows():
    #ds = row[0]
    #sample = row[1]
ds = 'DS4A'
sample = 'DS4A.1'
print(ds, sample)
# create output dir
ppi_website_dir = '/share/fsmresfiles/SpatialT/website/staging/data_v3_release/drug_screen/ppi_svg/'+ds+'/'+sample+'/'
if not os.path.exists(ppi_website_dir):
    os.makedirs(ppi_website_dir)
deg_dir = '/share/fsmresfiles/SpatialT/drug-target/'+ds+'/'+sample+'/DGE_dec_SVG/'
for deg_file in os.listdir(deg_dir):
    if 'UP' not in deg_file and 'DOWN' not in deg_file:
        # read in deg-svg file and filter (no svg fitler placed on cell types that don't have SVG analysis )
        cell_type = deg_file.split('.csv')[0]
        log2fc = pd.read_csv(deg_dir+deg_file)
        log2fc['abs_stat'] = [abs(i) for i in log2fc['stat'].tolist()]
        log2fc_top300 = log2fc.sort_values('abs_stat', ascending=False)[:300]
        log2fc_top300_sig = log2fc_top300[log2fc_top300['qval']<0.05]


        # map degs to inteactome
        ppi_index_interactome_top300 = []
        for i in clean_interactome.iterrows():
            row = i[1]
            if row['Protein_A_Name'] in log2fc_top300_sig['gene'].tolist() and row['Protein_B_Name'] in log2fc_top300_sig['gene'].tolist():
                ppi_index_interactome_top300.append(i[0])
        ppi_top300 = clean_interactome[clean_interactome.index.isin(ppi_index_interactome_top300)][['#Protein A','Protein B', 'Protein_A_Name','Protein_B_Name']]

        # drop ppis where both proteins are the same
        same_pro_index_top300 = []
        for i in ppi_top300.iterrows():
            row = i[1]
            if row['#Protein A'] == row['Protein B']:
                same_pro_index_top300.append(i[0])
        ppi_top300_nored = ppi_top300.drop(same_pro_index_top300)

        # format edges list for output
        ppi_top300_nored = ppi_top300_nored[['Protein_A_Name','Protein_B_Name']]
        ppi_top300_nored.columns = ['Source','Target']

        # format nodes list for output
        ppi_uniq_pro = list(set(ppi_top300_nored['Source'].tolist()) )
        for i in ppi_top300_nored['Target'].tolist():
            if i not in ppi_top300_nored['Source'].tolist() and i not in ppi_uniq_pro :
                ppi_uniq_pro.append(i)

        ppi_nodes_top300 = pd.DataFrame()
        ppi_nodes_top300['ID'] = ppi_uniq_pro
        ppi_nodes_top300['Label'] = ppi_uniq_pro

        ppi_nodes_top300 = pd.merge(ppi_nodes_top300, log2fc[['gene','stat']], how = 'left', right_on = 'gene', left_on = 'Label')
        ppi_nodes_top300 = ppi_nodes_top300.drop('gene',axis = 1)

        ppi_nodes_top300['sign'] = [np.sign(i) for i in ppi_nodes_top300['stat'].tolist()]
        ppi_nodes_top300['abs_stat'] = [abs(i) for i in ppi_nodes_top300['stat'].tolist()]

        # save
        ppi_nodes_top300.to_csv(ppi_website_dir+cell_type+'_ppi_nodes.csv', index=False)
        ppi_top300_nored.to_csv(ppi_website_dir+cell_type+'_ppi.csv', index=False)
        print(cell_type,'saved ppi')


DS4A DS4A.1
B.cell saved ppi
TMKI67 saved ppi
NK saved ppi
CD4 saved ppi
Fibroblast saved ppi
Endothelial saved ppi
T.reg saved ppi
DC saved ppi
Malignant saved ppi
CD8 saved ppi
Monocyte saved ppi
Macrophage saved ppi
